# Домашка 2

### Задание 1

### Надеюсь можно использовать numpy массивы для хранения element чтобы не реализовывать матричные операции)

In [469]:
import torch
import numpy as np

### Дореализовали все базовые операции. Теперь можем складывать / вычитать / делить / умножать элементы с другими элементами или со скалярами в любом порядке и возводить элемент в числовую степень и число в степень элемента

In [470]:
class Element:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Element(data={self.data}, grad={self.grad})"
    
    def __add__(self, other):
        if isinstance(other, Element):
            res = Element(self.data + other.data, (self, other), '+')
            def _backward():
                self.grad += 1 * res.grad
                other.grad += 1 * res.grad
            res._backward = _backward
            return res
        else:
            res = Element(self.data + other, tuple([self]), '+')
            def _backward():
                self.grad += 1 * res.grad
            res._backward = _backward
            return res
        
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        return self + other * (-1)
    
    def __rsub__(self, other):
        return other + (-1) * self
    
    def __neg__(self):
        return 0 - self

    def __mul__(self, other):
        if isinstance(other, Element):
            res = Element(self.data * other.data, (self, other), '*')
            def _backward():
                self.grad += other.data * res.grad
                other.grad += self.data * res.grad
            res._backward = _backward
            return res
        else:
            res = Element(self.data * other, tuple([self]), '*')
            def _backward():
                self.grad += other * res.grad
            res._backward = _backward
            return res
        
    def __rmul__(self, other):
        return self * other
        
    def __truediv__(self, other):
        if isinstance(other, Element):
            res = Element(self.data / other.data, (self, other), '/')
            def _backward():
                self.grad += 1 / other.data * res.grad
                other.grad += (-1) * self.data / other.data ** 2 * res.grad
            res._backward = _backward
            return res
        else:
            res = Element(self.data / other, tuple([self]), '/')
            def _backward():
                self.grad += 1 / other * res.grad
            res._backward = _backward
            return res
        
    def __rtruediv__(self, other):
        res = Element(other / self.data, tuple([self]), '/')
        def _backward():
            self.grad += (-1) * other/ self.data ** 2 * res.grad
        res._backward = _backward
        return res            
    
    def __pow__(self, power):
        res = Element(self.data ** power, tuple([self]), '**')
        def _backward():
            self.grad += power * self.data ** (power - 1) * res.grad
        res._backward = _backward
        return res
    
    def __rpow__(self, other):
        res = Element(other ** self.data, tuple([self]), '**')
        def _backward():
            self.grad += other ** self.data * np.log(other) * res.grad
        res._backward = _backward
        return res
    
    def relu(self):
        res = Element(max(0, self.data), (self,), 'ReLU')
        def _backward():
            self.grad += res.grad if res.data > 0 else 0
        res._backward = _backward

        return res

    def backward(self):
        visited = set()
        topo = []

        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)

        self.grad = 1
        for v in reversed(topo):
            v._backward()

# Функция которая из массива элементов делает массив чисел          
def numerize(arr:np.ndarray[Element]):
    new_arr = np.empty_like(arr)
    if len(arr.shape) == 1:
        for i in range(len(arr)):
            new_arr[i] = arr[i].data
        return new_arr
    else:
        for i in range(len(arr)):
            new_arr[i] = numerize(arr[i])
        return new_arr

# Функция которая из массива чисел делает массив элементов 
def elementize(arr:np.ndarray):
    new_arr = np.empty_like(arr, dtype=Element)
    if len(arr.shape) == 1:
        for i in range(len(arr)):
            new_arr[i] = Element(arr[i])
        return new_arr
    else:
        for i in range(len(arr)):
            new_arr[i] = elementize(arr[i])
        return new_arr

### BatchNorm

In [471]:
class BatchNorm1d:
    def __init__(self, dim, eps=1e-8):
        self.dim = dim
        # создаем параметры gamma и eps. Они Elements т.к. обучаются
        self.gamma = np.array([Element(data=1)] * dim)
        self.beta = np.array([Element(data=0)] * dim)
        self.eps = eps
        pass
    def __call__(self, x):
        # Вычисляем среднее и дисперсию
        # Применяем numerize потому что хотим интерпретировать их как числа
        mu = numerize(x.mean(axis=0))
        var = numerize(1 / (self.dim - 1) * np.sum((x - mu) ** 2, axis=0))
        # Нормализовываем и применяем преобразование
        x_normalized = (x - mu) / (var + self.eps) ** (1 / 2)
        y = self.gamma * x_normalized + self.beta
        return y        

### Пример

In [522]:
data = np.array([[Element(0), Element(1), Element(8)], [Element(5), Element(-7), Element(3)], 
                 [Element(-14), Element(6), Element(4)]])
data

array([[Element(data=0, grad=0), Element(data=1, grad=0),
        Element(data=8, grad=0)],
       [Element(data=5, grad=0), Element(data=-7, grad=0),
        Element(data=3, grad=0)],
       [Element(data=-14, grad=0), Element(data=6, grad=0),
        Element(data=4, grad=0)]], dtype=object)

### Все работает

In [523]:
norm = BatchNorm1d(3)
normalized_data = norm(data)
normalized_data

array([[Element(data=0.30460384952438446, grad=0),
        Element(data=0.15249857031487227, grad=0),
        Element(data=1.1338934182177578, grad=0)],
       [Element(data=0.8122769320650253, grad=0),
        Element(data=-1.067489992204106, grad=0),
        Element(data=-0.7559289454785052, grad=0)],
       [Element(data=-1.1168807815894097, grad=0),
        Element(data=0.9149914218892337, grad=0),
        Element(data=-0.3779644727392526, grad=0)]], dtype=object)

In [524]:
example_fun = 100 * np.sum(normalized_data ** 2 * np.array([1, -5, 4]))
example_fun

Element(data=-9.309175430516348e-07, grad=0)

### Градиенты считаются

In [525]:
example_fun.backward()
data

array([[Element(data=0, grad=6.1855670096715905),
        Element(data=1, grad=-23.255813948080043),
        Element(data=8, grad=342.85714236734697)],
       [Element(data=5, grad=16.49484535912424),
        Element(data=-7, grad=162.79069763656028),
        Element(data=3, grad=-228.57142824489796)],
       [Element(data=-14, grad=-22.68041236879583),
        Element(data=6, grad=-139.53488368848025),
        Element(data=4, grad=-114.28571412244898)]], dtype=object)

### И у параметров batchnorm тоже

In [526]:
display(norm.gamma, norm.beta)

array([Element(data=1, grad=-1.8618350452470622e-06),
       Element(data=1, grad=-1.8618350452470622e-06),
       Element(data=1, grad=-1.8618350452470622e-06)], dtype=object)

array([Element(data=0, grad=-5.684341886080802e-14),
       Element(data=0, grad=-5.684341886080802e-14),
       Element(data=0, grad=-5.684341886080802e-14)], dtype=object)

### Linear

In [477]:
class Linear:
    def __init__(self, in_features, out_features):
        # Параметры - матрица весов W и вектор сдвига b
        # инициализируем случайными весами от -0.5 до 0.5 
        self.W = elementize(np.random.rand(in_features, out_features) - 0.5)
        self.b = elementize(np.random.rand(out_features) - 0.5)
    def __call__(self, x):
        # Спасибо numpy за матричные операции!!
        return x @ self.W + self.b 

### Пример

In [551]:
data = np.array([[Element(0), Element(1), Element(8)], [Element(5), Element(-7), Element(3)], 
                 [Element(-14), Element(6), Element(4)]])
data

array([[Element(data=0, grad=0), Element(data=1, grad=0),
        Element(data=8, grad=0)],
       [Element(data=5, grad=0), Element(data=-7, grad=0),
        Element(data=3, grad=0)],
       [Element(data=-14, grad=0), Element(data=6, grad=0),
        Element(data=4, grad=0)]], dtype=object)

In [552]:
linear = Linear(3, 1)
result = linear(data)
result

array([[Element(data=-1.4332346403411869, grad=0)],
       [Element(data=-0.9533534851117019, grad=0)],
       [Element(data=-0.3812764872122557, grad=0)]], dtype=object)

In [553]:
example_loss = np.sum((result - np.array([[-1, -1, -1]]))**2)
example_loss

Element(data=1.7180608086575633, grad=0)

In [554]:
example_loss.backward()
linear.W

array([[Element(data=0.022777556690676493, grad=-50.57337962752157)],
       [Element(data=0.09456572362960702, grad=17.71548499300315)],
       [Element(data=-0.22450383216259173, grad=-5.1062611614817435)]],
      dtype=object)

In [585]:
class Dropout:
    def __init__(self, p=0.2):
        self.p = p
    def __call__(self, x):
        # Строки - батчи, столбцы - нейроны. Мы генерируем строку 0 или 1 с вероятностью p
        # умножив x на эту строку мы "отключим" каждый нейрон с вероятностью p
        mask = (np.random.rand(x.shape[1]) > self.p).astype(int)
        return x * mask

In [592]:
data = np.array([[Element(0), Element(1), Element(8)], [Element(5), Element(-7), Element(3)], 
                 [Element(-14), Element(6), Element(4)]])
data

array([[Element(data=0, grad=0), Element(data=1, grad=0),
        Element(data=8, grad=0)],
       [Element(data=5, grad=0), Element(data=-7, grad=0),
        Element(data=3, grad=0)],
       [Element(data=-14, grad=0), Element(data=6, grad=0),
        Element(data=4, grad=0)]], dtype=object)

### Видим что все работает

In [602]:
dropout = Dropout(0.3)
result = dropout(data)
result

array([[Element(data=0, grad=0), Element(data=0, grad=0),
        Element(data=8, grad=0)],
       [Element(data=5, grad=0), Element(data=0, grad=0),
        Element(data=3, grad=0)],
       [Element(data=-14, grad=0), Element(data=0, grad=0),
        Element(data=4, grad=0)]], dtype=object)

### ReLU

In [485]:
class ReLU:
    def __call__(self, x):
        # создаем выход того же размера
        res = np.empty_like(x, dtype=Element)
        if len(x.shape) == 1:
            for i in range(len(x)):
                # дергаем relu у элемента
                res[i] = x[i].relu()
            return res
        else:
            for i in range(len(x)):
                res[i] = self(x[i])
            return res

In [486]:
data = np.array([[Element(0), Element(1), Element(8)], [Element(5), Element(-7), Element(3)], 
                 [Element(-14), Element(6), Element(4)]])
data

array([[Element(data=0, grad=0), Element(data=1, grad=0),
        Element(data=8, grad=0)],
       [Element(data=5, grad=0), Element(data=-7, grad=0),
        Element(data=3, grad=0)],
       [Element(data=-14, grad=0), Element(data=6, grad=0),
        Element(data=4, grad=0)]], dtype=object)

### Отрицательные элементы обнулились

In [487]:
relu = ReLU()
result = relu(data)
result

array([[Element(data=0, grad=0), Element(data=1, grad=0),
        Element(data=8, grad=0)],
       [Element(data=5, grad=0), Element(data=0, grad=0),
        Element(data=3, grad=0)],
       [Element(data=0, grad=0), Element(data=6, grad=0),
        Element(data=4, grad=0)]], dtype=object)

### Sigmoid

In [488]:
# проверка что все действительно правильно дифференцируется
a = Element(2)
res = 1 / (1 + np.e ** (-a))
res.backward()
a
# ответ правильный

Element(data=2, grad=0.1049935854035065)

In [489]:
def sigmoid(x):
    return 1 / (1 + np.e ** (-x))

# как ReLU только вызываем сигмоиду
class Sigmoid:
    def __call__(self, x):
        y = np.empty_like(x)
        if len(x.shape) == 1:
            for i in range(len(x)):
                y[i] = sigmoid(x[i])
            return y
        else:
            for i in range(len(x)):
                y[i] = self(x[i])
            return y

In [491]:
data = np.array([[Element(0), Element(1), Element(8)], [Element(5), Element(-7), Element(3)], 
                 [Element(-14), Element(6), Element(4)]])
data

array([[Element(data=0, grad=0), Element(data=1, grad=0),
        Element(data=8, grad=0)],
       [Element(data=5, grad=0), Element(data=-7, grad=0),
        Element(data=3, grad=0)],
       [Element(data=-14, grad=0), Element(data=6, grad=0),
        Element(data=4, grad=0)]], dtype=object)

### Значения стали от нуля до единицы

In [492]:
sigm = Sigmoid()
result = sigm(data)
result

array([[Element(data=0.5, grad=0),
        Element(data=0.7310585786300049, grad=0),
        Element(data=0.9996646498695336, grad=0)],
       [Element(data=0.9933071490757153, grad=0),
        Element(data=0.0009110511944006456, grad=0),
        Element(data=0.9525741268224331, grad=0)],
       [Element(data=8.315280276641327e-07, grad=0),
        Element(data=0.9975273768433653, grad=0),
        Element(data=0.9820137900379085, grad=0)]], dtype=object)

### Softmax

In [494]:
def softmax(x):
    return np.e ** x / np.sum(np.e ** x)
    

class Softmax:
    def __call__(self, x):
        # построчно вызываем softmax
        res = np.empty_like(x)
        for i in range(len(x)):
            res[i] = softmax(x[i])
        return res
        

In [495]:
data

array([[Element(data=0, grad=0), Element(data=1, grad=0),
        Element(data=8, grad=0)],
       [Element(data=5, grad=0), Element(data=-7, grad=0),
        Element(data=3, grad=0)],
       [Element(data=-14, grad=0), Element(data=6, grad=0),
        Element(data=4, grad=0)]], dtype=object)

### Сумма по столбцам стала равна 1

In [496]:
soft = Softmax()
res = soft(data)
res

array([[Element(data=0.0003350447116928156, grad=0),
        Element(data=0.0009107459515158803, grad=0),
        Element(data=0.9987542093367913, grad=0)],
       [Element(data=0.8807923113022759, grad=0),
        Element(data=5.411774999819953e-06, grad=0),
        Element(data=0.11920227692272425, grad=0)],
       [Element(data=1.815458084611523e-09, grad=0),
        Element(data=0.8807970763788322, grad=0),
        Element(data=0.11920292180570966, grad=0)]], dtype=object)

In [497]:
np.sum(res, axis=1)

array([Element(data=1.0, grad=0),
       Element(data=0.9999999999999999, grad=0),
       Element(data=1.0, grad=0)], dtype=object)

In [604]:
a = [1, 2]
b = [3, 4]

In [607]:
a += b

In [608]:
a

[1, 2, 3, 4]